#[1] Mount Google Drive

In [ ]:
import os
import sys
# Set Path
from google.colab import drive
drive.mount('/content/drive')



#[2] Setting up Environment and Install Requirements

In [219]:
path_wd = '/content/drive/MyDrive/Github/Content'
os.chdir(path_wd)

!pip install -r sources/ChinaFilm/requirements.txt
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import pandas as pd
import numpy as np
import datetime as dt
import re

# [3] Load Release and Registration Records

In [385]:
path_regs = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm/records/contents_of_registrations.csv'
path_releases = '/content/drive/MyDrive/Github/Content/sources/ChinaFilm/records/contents_of_releases.csv'
df_releases = pd.read_csv(path_releases, index_col=0, encoding='utf-8')
df_reg = pd.read_csv(path_regs, index_col=0, encoding='utf-8')

# Get Latest Issue
str_date_curr_release = df_releases.sort_values(by='公示日期', ascending=False).iloc[0]['公示日期']
dt_curr_release = dt.datetime.strptime(str_date_curr_release, '%Y-%m-%d')
df_curr_releases = df_releases[df_releases['公示日期']==date_curr_release]
df_curr_releases = df_curr_releases[['电影类别', '编码', '公映证号', '片名', '第一出品单位']].copy()
df_curr_releases['匹配片名'] = df_curr_releases['片名'].apply(lambda x: mmatch[x] if x in mmatch else x)

In [386]:
df_matched = df_reg[df_reg['片名'].isin(df_curr_releases['匹配片名'].to_numpy())]
df_matched = df_matched[['备案立项号', '片名', '备案单位', '编剧', '备案结果', '备案地', '梗概', '公示日期']].copy()
df_matched.rename(columns={'公示日期':'备案公示日期'}, inplace=True)
df_c = df_curr_releases.merge(df_matched, on='匹配片名', how='left')
df_c['备案公示日期'] = df_c['备案公示日期'].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
df_c['备案至今'] = dt_curr_release - df_c['备案公示日期']
df_c

KeyError: ignored

In [378]:
df_curr_releases.loc[~df_curr_releases['片名'].isin(df_matched['片名'].to_numpy()),'片名']

,电影类别,编码,公映证号,片名,第一出品单位
序号,,,,,
2,国产故事片,1100312021,32,如歌少年,浙江思成影业有限公司
4,国产故事片,1100452021,44,永春拳之黄金蝴蝶刀,广东镇宇影视传媒有限公司
9,国产故事片,1100782021,69,射雕英雄传之九阴白骨瓜,北京拉近影业有限公司
13,国产故事片,1101292021,120,毒影迷踪,陕西新弘晟晖影视制作发行有限公司
14,国产故事片,1101302021,121,伊水栾山,河南电影电视制作集团有限公司
16,国产故事片,1101722021,150,谢谢你温暖我的世界,河南日报报业集团有限公司
18,国产故事片,1101782021,161,澳门十月,伟雄文化传媒（佛山）有限公司
20,国产故事片,1102122021,186,超越,上海猎豹文化传媒有限公司
21,国产故事片,1102192021,197,守岛人,八一电影制片厂


In [376]:
mmatch = {
          '如歌少年':'你是谁的孩子',
          '永春拳之黄金蝴蝶刀':'永春',
          '射雕英雄传之九阴白骨爪':'射雕英雄传之九阴白骨瓜',
          '毒影迷踪':'捉拿归案',
          '伊水栾山':'马海明',
          '谢谢你温暖我的世界':'在月季的故乡',
          '澳门十月':'澳门第一个十月',
          '超越':'极速超越',
          '守岛人': '两个人的海岛',
          '来处是归途':'又一夏',
          '山里山外':'山外',
          '射雕英雄传之降龙十八掌': '射雕英雄传',
          
          }

In [333]:
df = df_c.dropna().copy()
idx_to_drop = [7]
df.drop(idx_to_drop, inplace=True)

tmp = df[['备案单位','第一出品单位']].apply(lambda x: re.search(x['第一出品单位'], x['备案单位']), axis=1).to_frame()
idx_co_change = tmp[tmp[0].isna()].index
df['公司变更'] = 0
df.loc[idx_co_change, '公司变更'] = 1
df_view = df[['片名',  '梗概', '备案至今', '备案结果', '第一出品单位', '备案单位', '公司变更']].copy()
df_view.loc[df_view['公司变更'] == 0, '备案单位'] = ''
df_view.rename(columns={'备案单位':'原备案单位'}, inplace=True)
df_view.loc[df_view['备案结果'].str.contains('修改'),'备案结果'] = '是'
df_view.loc[df_view['备案结果'].str.contains('同意'),'备案结果'] = ''
df_view.rename(columns={'备案结果':'是否修改'}, inplace=True)
df_view['备案至今'] = round((df_view['备案至今'].dt.days + 60) / 365, 1)
df_view.sort_values(by='备案至今', ascending=False, inplace=True)
df_view['备案至今'] = df_view['备案至今'].apply(lambda x: '> ' + str(x) + ' 年')
df_view.drop('公司变更', axis=1, inplace=True)
df_view

,片名,梗概,备案至今,是否修改,第一出品单位,原备案单位
30,阳光姐妹淘,曾经是中学sunny小团体成员，25年后为人妇为人母的美君，偶然重逢了当年的挚友静华，为帮身...,> 4.7 年,,上海喜焰文化发展有限公司,天津逗友文化传播有限公司
23,多乐路,一场交通事故令木森、大海、平安和一对情侣有了交集。在这平行时空中，每个人的人生千差万别、欢喜哀怨。,> 4.5 年,,美亚长城传媒（北京）有限公司,
22,丑角,一个女子投靠到乡间剧团，受到团长精心照顾，并渐渐在乡间走红。出于感激和依赖，与团长走在一起。...,> 4.0 年,,北京活动影像国际广告有限公司,
6,越界,摄影师志成有着一个多年的秘密，从18岁起，他时常觉得被“跟踪”，在目睹了一场意外后，他更确信...,> 3.8 年,,深圳市乐享影业有限公司,
24,等儿的湿地,姥姥去世了，晓涵来到姥姥当知青下乡时的三江湿地，意外发现姥姥留下的明信片，得知东方白鹳“等儿...,> 3.8 年,,北京知梦文化影视传媒有限公司,
10,奇迹小子,王汉坤为实现自己心中的梦想让花式摩托车极限运动走上国际赛场，对六岁的儿子王克进行残酷的魔鬼训...,> 3.2 年,,安徽天惟影视文化有限公司,
5,热带往事,1998年夏夜，中国南方的一个小城市意外全城停电，一起命案也因此被掩盖在黑暗之中。车祸肇事者...,> 3.1 年,是,坏猴子（上海）文化传播有限公司,
12,六日破局,航天基地张教授的女儿张艺雯外出找其男友戴维后失联，张教授四处寻找之际，绑匪发来信息，要求张教...,> 2.6 年,,陕西新弘晟晖影视制作发行有限公司,西安亚细亚影视制作发行有限公司
11,北海屠龙记,因参与屠龙之战而转生的蜀山弟子沈秀，为寻爷爷复生之机，而引魔龙觉醒，少年沈秀将再次与魔龙决战...,> 2.3 年,,江苏东峰文化发展有限公司,
19,完美受害人,单身母亲夏美莲终于摆脱家暴，带着年幼的女儿来到新城市重建生活，却饱受创伤后应激障碍以及各种幻...,> 2.2 年,,北京华诚传媒有限公司,


# [4] Generate Article

##[4.1]标题：影片的前世今生

In [305]:
# Title
T = ''

2021

##[4.2] 段落

In [308]:
P1 ='{Y}年{M}月{D}日,'.format(Y=dt_curr_release.year, M=dt_curr_release.month, D=dt_curr_release.day)
P1 += '又一批电影历经千辛万苦，获得了出生证。'

P1

'2021年6月4日,又一批电影历经千辛万苦，获得了出生证。'

In [374]:
df_reg[df_reg['备案单位'].str.contains('先施')]

,备案立项号,片名,备案单位,编剧,备案结果,备案地,梗概,公示日期,公示批次名称,备案详细页链接,公示批次链接
电影类别,,,,,,,,,,,
故事影片,影剧备字[2018]第4325号,你是谁的孩子,杭州金沙滩影视传媒有限公司、浙江先施影业有限公司,施炳权、李赟,同意拍摄,浙江省,六年级学生田行健同母亲之间有着隔阂。出生单亲家庭，因父亲的因素母亲耿耿于怀，拒绝田行健接触音...,2018-08-28,国家电影局关于2018年08月（上旬）全国电影剧本（梗概） 备案、立项公示的通知-电影备案...,http://www.chinafilm.gov.cn/chinafilm/utils/se...,http://www.chinafilm.gov.cn/chinafilm/contents...


In [336]:
df_curr_releases.reset_index()

,序号,电影类别,编码,公映证号,片名,第一出品单位
0,1,国产故事片,1100132021,13,海峡情缘,泉州市中之台文化传播有限公司
1,2,国产故事片,1100312021,32,如歌少年,浙江思成影业有限公司
2,3,国产故事片,1100442021,42,加油兄弟,山西星道影视文化传媒有限公司
3,4,国产故事片,1100452021,44,永春拳之黄金蝴蝶刀,广东镇宇影视传媒有限公司
4,5,国产故事片,1100552021,54,深宅迷案,西安识君影视文化有限公司
5,6,国产故事片,1100582021,57,热带往事,坏猴子（上海）文化传播有限公司
6,7,国产故事片,1100592021,58,越界,深圳市乐享影业有限公司
7,8,国产故事片,1100632021,62,兰湖水畔,兰溪市芥子园文化传媒有限公司
8,9,国产故事片,1100782021,69,射雕英雄传之九阴白骨瓜,北京拉近影业有限公司
9,10,国产故事片,1100952021,91,奇迹小子,安徽天惟影视文化有限公司


# [5] Output Word Doc

In [298]:
!pip install python-docx
!pip install lxml
from docx import Document
from docx.shared import Inches
#from docx.text.parargaph import Paragraph

     |████████████████████████████████| 5.6MB 5.1MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-cp37-none-any.whl size=184508 sha256=46528d7cc3f990b06653ea4f7f1b875c8de4064b3c8ad9b7c8a409fddec15b61
  Stored in directory: /root/.cache/pip/wheels/a6/90/f1/a7cb70b38633ae04e7fb963b1c70f63fd6fc01c075b8230adc
Successfully built python-docx
